# Example 6: Parkhurst and Appelo 2013 (PHREEQC-3 Example 11) 3D variant of Cation Exchange with DISV grid

This notebook demonstrates a 3D varriant of the MF6RTM benchmark Example 4 and also utilizes the DISV package for the MODFLOW 6 model grid. The example was originally used as PHREEQM (Nienhuis et al., 1994) test case, and is also included in the PHREEQC-3 documentation (Parkhurst and Appelo, 2013) as Example 11. Further discussion can be found in (Appelo and Postma, 1993), where it forms Example 10.13, and in (Appelo, 1994). The one-dimensional simulation problem describes a hypothetical column experiment where porewater containing sodium (Na<sup>+</sup>), potassium (K<sup>+</sup>) and nitrate (NO<sup>-</sup><sub>3</sub>) in equilibrium with exchangeable cations is flushed by a calcium chloride (CaCl<sub>2</sub>) solution.

The workflow for this example starts with loading a pre-existing MODFLOW 6 model, then builds the PHREEQC yaml file from scratch based off of the MODFLOW 6 input file parameters. The input files required for the MODFLOW 6 model are located in the ex6 directory.   

# Installation and Setup

This notebook is designed to be run from:   
- a custom [conda](https://docs.conda.io/en/latest/) virtual environment as described in our [Install Development Environment](benchmark/readme.md) instructions, especially including:
  - Creating a custom virtual environment using the `environment.yml` file included in this folder (step 3)
  - Adding this repository to your conda path (step 4)

## Python Imports

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image

import flopy
import phreeqcrm

In [ ]:
# Import the MF6RTM package
import mf6rtm

### If you get `ModuleNotFoundError`

If you get `ModuleNotFoundError`, you need to install `mf6rtm` into your environment.

If using the development environment, the following steps will install in develop mode:
1. Run the [`conda develop`](https://docs.conda.io/projects/conda-build/en/latest/resources/commands/conda-develop.html) command in your terminal with your local absolute path to the `src` directory of this repo.
2. Restart the kernel.
3. Rerun the import statements above.

In [ ]:
# Confirm conda environment
env = os.environ['CONDA_DEFAULT_ENV']
env

In [ ]:
# Find project directory (i.e. the parent to current working directory for this notebook)
project_path = Path.cwd().parent

# Your source directory should be: 
mf6rtm_source_path = project_path/ 'src'
print(mf6rtm_source_path)
mf6rtm_source_path.exists()

### To install in developer mode:
- Confirm that the output of `src_path` points to the MF6RTM `src` directory 
- Uncomment and run the line below.
- NOTE 1: The Jupyter [`%conda` magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-conda) runs conda terminal commands directly from this notebook.
- NOTE 2: We can inject Python string objects into magic commmands using a syntax similar to Python's F-strings.

In [ ]:
# %conda develop {mf6rtm_source_path}

If the path was added, Restart the kernel and rerun the cells above.

## Set Paths to Input and Output Files with `pathlib` 

Use the [pathlib](https://docs.python.org/3/library/pathlib.html) library (built-in to Python 3) to manage paths indpendentely of OS or environment. See this [blog post](https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f) to learn about the many benefits over using the `os` library.

In [ ]:
# Find your current working directory, which should be folder for this notebook.
working_dir = Path.cwd()
working_dir

In [ ]:
# Set the simulation name
sim_name = 'ex6'

In [ ]:
# Path to MF6 inputs and outputs
mf6_input_path = working_dir / sim_name 
mf6_input_path

In [ ]:
# Phreeqc input file folders
dataws = working_dir / 'data'
databasews = working_dir / 'database'

# Write/Load Modflow 6 Model Simulation Info using Flopy
To set up PhreeqcRM and MF6RTM simulation objects. Creates Modflow 6 input files, if they don't already exit. 

In [ ]:
# Set the name or path of the Modflow 6 executable
if env == 'mf6rtm_demo':
    print('Using mf6 executable from conda environment')
    mf6_exe = 'mf6'
else:
    mf6_exe = Path(r'C:/program files/gms 10.8 64-bit/python/lib/site-packages/xms/executables/modflow6/mf6.exe')
    print('mf6_exe.exists() = ',mf6_exe.exists())

In [ ]:
# Path to simulation workspace
sim_ws = mf6_input_path

In [ ]:
# Functions for writting and running the Modflow 6 input files

def build_models(name, inflow):
    print(f"Building model...{name}")
    
    sim = flopy.mf6.MFSimulation(sim_name=name, sim_ws=sim_ws, exe_name=mf6_exe)
    
    tdis_rc = []
    for i in range(nper):
        tdis_rc.append((perlen, nstp, 1.0))
    flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_rc, time_units=time_units)
    
    ### Begin GWF
    gwf_name = "gwf_" + name
    
    gwf = flopy.mf6.ModflowGwf(sim, modelname=gwf_name, save_flows=True)
    imsgwf = flopy.mf6.ModflowIms(
        sim,
        print_option="ALL",
        complexity="complex",
        outer_dvclose=hclose,
        outer_maximum=nouter,
        under_relaxation="NONE",
        inner_maximum=ninner,
        inner_dvclose=hclose,
        rcloserecord=rclose,
        linear_acceleration="CG",
        scaling_method="NONE",
        reordering_method="NONE",
        relaxation_factor=relax,
        filename=f"{gwf.name}.ims")
    sim.register_ims_package(imsgwf, [gwf.name])

    nlay = 2
    nrow = 2
    ncol = 40
    delc = 0.5 # (m)
    delr = 0.01 #Lx/ncol (m)
    Lx = ncol*delr #0.08 (m)
    top = 1. # (m)
    zbotm = 0. # (m)
    botm = np.linspace(top,zbotm,nlay + 1)[1:]
    ncpl = ncol*nrow

    # load in grid info from csv files
    vertices_csv = np.genfromtxt(v_file_path, delimiter=',', names=True, dtype=None, encoding='utf-8')
    vertices_recdata = vertices_csv.view(np.recarray)
    cell2d_csv = np.genfromtxt(c_file_path, delimiter=',', names=True, dtype=None, encoding='utf-8')
    cell2d_recdata = cell2d_csv.view(np.recarray)
    
    nvert = len(vertices_recdata)

    disv = flopy.mf6.ModflowGwfdisv(
        gwf,
        nlay=nlay,
        ncpl=ncpl,
        nvert=nvert,
        top=top,
        botm=botm,
        vertices=vertices_recdata,
        cell2d=cell2d_recdata,
        filename = f"{gwf.name}.disv"
        )
    
    # get propeties for gwt disv
    nlay_gwf = gwf.disv.nlay.array
    ncpl_gwf = gwf.disv.ncpl.array 
    top_gwf = gwf.disv.top.array
    botm_gwf = gwf.disv.botm.array
    nverts_gwf = gwf.disv.nvert.array
    vertices_gwf = gwf.disv.vertices.array
    cell2d_gwf = gwf.disv.cell2d.array
    
    npf = flopy.mf6.ModflowGwfnpf(
        gwf,
        save_specific_discharge=True,
        icelltype=1,
        k=hydraulic_conductivity)
    
    icgwf = flopy.mf6.ModflowGwfic(gwf, strt=initial_heads)

    chdspd = [[(0,39),chd_head],[(0,79),chd_head],[(1,39),chd_head],[(1,79),chd_head]]  
    chd = flopy.mf6.ModflowGwfchd(
        gwf,
        stress_period_data=chdspd,
        pname="CHD-1")

    wel_cell_idx = [(0,0),(0,40),(1,0),(1,40)]
    welspd = []
    for c in range(len(wel_cell_idx)):
        welspd.append([wel_cell_idx[c],inflow,*inflow_conc])
    auxiliary_vars = [f"CONCENTRATION_{solute}" for solute in solute_names]
    wel = flopy.mf6.ModflowGwfwel(
        gwf,
        stress_period_data=welspd,
        pname="WEL-1",
        auxiliary=auxiliary_vars)
    
    head_filerecord = f"{name}.hds"
    budget_filerecord = f"{name}.bud"
    ocgwf = flopy.mf6.ModflowGwfoc(
        gwf,
        head_filerecord=head_filerecord,
        budget_filerecord=budget_filerecord,
        saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")])
    ### End GWF
    
    ### Begin GWT
    for c in range(len(solute_names)):
        gwt_name = 'gwt_' + solute_names[c]

        gwt = flopy.mf6.ModflowGwt(sim, modelname=gwt_name)
        imsgwt = flopy.mf6.ModflowIms(
            sim,
            print_option="ALL",
            outer_dvclose=hclose,
            outer_maximum=nouter,
            under_relaxation="NONE",
            inner_maximum=ninner,
            inner_dvclose=hclose,
            rcloserecord=rclose,
            linear_acceleration="BICGSTAB",
            scaling_method="NONE",
            reordering_method="NONE",
            relaxation_factor=relax,
            filename=f"{gwt.name}.ims")
    
        sim.register_ims_package(imsgwt, [gwt.name])

        disvgwt = flopy.mf6.ModflowGwtdisv(
            gwt,
            length_units=length_units,
            nlay=nlay_gwf,
            ncpl=ncpl_gwf,
            nvert=nverts_gwf,
            top=top_gwf,
            botm=botm_gwf,
            vertices=vertices_gwf,
            cell2d=cell2d_gwf,
        )
        mst = flopy.mf6.ModflowGwtmst(
            gwt, 
            porosity=porosity,
            first_order_decay=None)
        
        icgwt = flopy.mf6.ModflowGwtic(gwt, strt=ic_conc_l[c])
        
        adv = flopy.mf6.ModflowGwtadv(gwt, scheme="tvd")
        
        dsp = flopy.mf6.ModflowGwtdsp(
            gwt,
            xt3d_off=True,
            alh=dispersivity,
            ath1=dispersivity*0.1,
            atv=dispersivity*0.1)
        
        sourcerecarray = [("WEL-1", "AUX", "CONCENTRATION_"+solute_names[c])]
        ssm = flopy.mf6.ModflowGwtssm(gwt, sources=sourcerecarray)
        
        ocgwt = flopy.mf6.ModflowGwtoc(
            gwt,
            budget_filerecord=f"{gwt.name}.cbc", 
            concentration_filerecord=f"{gwt.name}.ucn",
            concentrationprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
            saverecord=[("CONCENTRATION", "ALL")])
        
        gwfgwt = flopy.mf6.ModflowGwfgwt(
            sim, exgtype="GWF6-GWT6", exgmnamea=gwf.name, exgmnameb=gwt.name,
            filename='gwfgwt_'+solute_names[c]+'.gwfgwt')
        
    ### End GWT
    
    return sim

def write_models(sim, silent=True):
    
    ''' writes modflow 6 input files'''
    
    sim.write_simulation(silent=silent)

def run_models(sim, silent=True):
    
    '''runs modflow 6 simulation'''
    
    success, buff = sim.run_simulation(silent=silent)
    assert success, buff


In [ ]:
# checks to see if Modflow 6 input files already exist
sim_nam_file_path = sim_ws / "mfsim.nam"

if sim_nam_file_path.exists():
    sim = flopy.mf6.MFSimulation.load(
        sim_ws=sim_ws,
        exe_name=mf6_exe,#'mf6',
        verbosity_level=0,
    )
else:
    # additional parameters required for creating Modflow 6 input files
    length_units = "meters"
    time_units = "days"
    nper = 1  # Number of periods
    tstep = 0.001 #0.002 *0.5 # timestep (days)
    perlen = 0.24 #nper * [time_end / nper]  # Simulation time length (d)
    nstp = perlen/tstep  # Number of time steps    

    hydraulic_conductivity = 1.  # Hydraulic conductivity (m/d)
    porosity = 1  # porosity (unitless)
    diffusion_coefficient = 0.57024  # diffusion coefficient (m^2/d)
    dispersivity = 0.002 # Longitudinal dispersivity (m) 

    initial_heads = 1.
    solute_names = [
        'H', 'O', 'Ch', 'Ca', 
        'Cl', 'K', 'N', 'Na',
    ]
    inflow_conc  = [   # units = mol/L 
        110684.00, 55342.0856, -0.0000013109877, 0.59822581,
        1.19645161, 0.0000, 0.0000, 0.0000,
    ] 
    nsolutes = len(solute_names)
    # initial concentration of each solute in solute_names
    ic_conc_l = [       # units = mol/L
        110684.1711486, 55345.67494365, -0.000000337912141, 0.0,
        0.0, 0.19940729, 1.19645161, 0.99703984,
    ] 

    chd_head = 1. # (m)
    inflow = 0.25 # flow rate into column m3/d

    nouter, ninner = 100, 300
    hclose, rclose, relax = 1e-9, 1e-6, 0.97 

    # full path to disv package verticies .csv file 
    v_file_path = sim_ws / "3Dncol40_vert.csv"
    # full path to disv package cell2d .csv file 
    c_file_path = sim_ws / "3Dncol40_cell.csv"

    # create simulation and write Modflow 6 input files
    sim = build_models(sim_name, inflow)
    write_models(sim, silent=False)

In [ ]:
# view Modflow 6 models within the simulation
sim.model_names

## Read GWT Model Names & Initial Conditions

In [ ]:
# get gwt components and initial conditions
ic_dict = {}
# NOTE: this assumes names of gwt models are the component names 
for mname in sim.model_names:
    m = sim.get_model(mname)
    if m.model_type == 'gwt6':
        # get initial conditions
        assert hasattr(m, 'ic'), f"GWT model '{m.name}' is missing an 'ic' package."
        ic_dict[m.name] = m.ic.strt.array
        
gwt_model_names = list(ic_dict.keys())
gwt_model_names

### Create Chem Component: GWT Model Dictionary

In [ ]:
components = ['H', 'O', 'Charge', 'Ca', 'Cl', 'K', 'N', 'Na']
component_gwt_model_dict = dict(zip(components, gwt_model_names))
component_gwt_model_dict

## Read Time Step Info

In [ ]:
# Get time discretization info from the `tdis` package
tdis = sim.tdis
nper = tdis.nper.get_data()          # number of stress periods
perioddata = tdis.perioddata.get_data()
nstp = perioddata['nstp']            # number of timesteps per stress period
perlen = perioddata['perlen']        # lenght of stress periods
tsmult = perioddata['tsmult']        # timestep multiplier
t_units = tdis.time_units.get_data() # units

print(nper, nstp, perlen, tsmult, t_units)

## Read Stress Period Info

In [ ]:
# get boundary condition packages with transport
# read in groundwater flow model
gwf_name = "gwf_" + sim_name
gwf_model = sim.get_model(gwf_name)
# for well package (wel)
wel = gwf_model.get_package('wel')
if wel.has_stress_period_data == True:
    spd_wel_dict = wel.stress_period_data.get_data()
    print(spd_wel_dict)

In [ ]:
# data stored in numpy record arrays
spd_wel_dict[0].dtype.fields.keys()

In [ ]:
spd_wel_dict[0]['cellid']

NOTE: `cellid` is the cell identifier, and depends on the type of grid that is used for the simulation. 
- For a structured grid that uses the DIS input file, CELLID is the layer, row, and column. 
- For a grid that uses the DISV input file, CELLID is the layer and CELL2D number. 
- If the model uses the unstructured discretization (DISU) input file, CELLID is the node number for the cell.

In [ ]:
# injection rate
# q = 1 #injection rate m3/d
spd_wel_dict[0]['q']

### Create Chem Component: Auxilary Variable Dictionary

In [ ]:

components

In [ ]:
spd_wel_dict

In [ ]:
# Create dictionary of auxiliary variables for spress period package
welchem_dict = {}
component_spd_auxvar_dict = {}
auxvar_prefix = 'concentration_'
for n in range(0,nper):
    for component in components:
        component_code = component.lower()
        if component_code == 'charge':
            component_code = 'ch'
        welchem_dict[component] = spd_wel_dict[n][f'{auxvar_prefix}{component_code}']
        component_spd_auxvar_dict[component] = f'{auxvar_prefix}{component_code}'
welchem_dict

In [ ]:
component_spd_auxvar_dict

# Run Modflow 6 simulation only

In [ ]:
run_models(sim, silent=False)
#sim.run_simulation(silent=False)

## Plot MF6 Transport Results with no Reactions

When just running MF6, before any coupling.

NOTE: `conc = sim.get_model(gwt_model_name).output.concentration().get_alldata()` returns:  

```python
conc[0].shape = 
    (240, 2, 1, 80)
#   ^     ^  ^  ^
#   |     |  |  number of cells per layer (ncpl = 80)
#   |     |  dummy row dimension (always 1 for DISV)
#   |     number of layers (nlay = 2)
#   number of time steps (240)
```


In [ ]:
# Plot concentration results timeseries at last cell 
# for conservative species Chloride
component = 'Cl'
gwt_model_name = component_gwt_model_dict[component]
conc = sim.get_model(gwt_model_name).output.concentration().get_alldata()
conc.shape

In [ ]:
# Get times
times_list = sim.get_model(gwt_model_name).output.concentration().get_times()
times = np.array(times_list)
times.shape

In [ ]:
mf6_results_df = pd.DataFrame(
    conc[:,0,0,-1], # select timeseries at last cell
    index=times,
    columns=[component],
)
mf6_results_df.plot(xlabel='time (d)', ylabel='Concentration (mol/L)')

In [ ]:
# Confirm breakthrough curves for other components mirror Cl
components = ['Ca', 'K']
for component in components:
    gwt_model_name = component_gwt_model_dict[component]
    conc = sim.get_model(gwt_model_name).output.concentration().get_alldata()
    print(sim_name, component, gwt_model_name, conc.shape)
    fig = plt.figure(figsize=(18, 5))
    plt.plot(times, conc[:,0,0,-1])

# Create PhreeqcRM YAML config file from scratch

Based on the [`phreeqcrm/swig/python/ex11-advect/ex11-advect.ipynb`](https://github.com/usgs-coupled/phreeqcrm/blob/master/swig/python/ex11-advect/ex11-advect.ipynb) example notebook

In [ ]:
chemistry_name_filepath = dataws / 'ex6_advect.pqi'
chemistry_name_filepath.exists()

## Initialize YAML file and set grid size

In [ ]:
# Create YAMLPhreeqcRM document object
yrm = phreeqcrm.YAMLPhreeqcRM()

In [ ]:
# Number of grid cells, read in from the Modflow 6

# Read the grid type / discretization package name
for package_name in gwf_model.package_names:
    if 'dis' in package_name: 
        grid_package_name = package_name

# Get spatial discretization info, for DISV
grid_package = gwf_model.get_package(grid_package_name)
nlay = grid_package.nlay.get_data()  # number of layers
ncpl = grid_package.ncpl.get_data()  # number of cells per layer
print(nlay, ncpl)

# Calculate total number of grid cells
nxyz = nlay * ncpl
nxyz

In [ ]:
# Set GridCellCount
yrm.YAMLSetGridCellCount(nxyz)  
    # Number of cells for the PhreeqcRM instance

## Set Properties

In [ ]:
# Set these parameters here, 
# to match order in `Mup3d._write_phreeqc_init_file()`
yrm.YAMLThreadCount(1)
yrm.YAMLSetComponentH2O(False)
yrm.YAMLUseSolutionDensityVolume(False)

In [ ]:
phreeqc_output_prefix = mf6_input_path / '_phreeqc'
phreeqc_output_prefix

In [ ]:
# From `Mup3d._write_phreeqc_init_file()`

# Open files for phreeqcrm logging
yrm.YAMLSetFilePrefix(str(phreeqc_output_prefix))
yrm.YAMLOpenFiles()

In [ ]:
yrm.YAMLSetErrorHandlerMode(1)   
    # 0 (default), return to calling program with an error return code; 
    # 1, throw an exception; 2, attempt to exit gracefully
# yrm.YAMLSetComponentH2O(False)              
    # True (default), excess H, excess O, and water are included in the 
    #   component list (require less accuracy in transport); 
    # False, total H and O are included in the component list (requires 
    #   transportresults to be accurate to eight or nine significant 
    #   digits)
yrm.YAMLSetRebalanceFraction(0.5)           
    # 0.5 (default); 
    # 0, eliminate load rebalancing; 
    # 1, maximum load rebalancing setting; 
    # Less than 1 recommended to avoid too many cells transferred at one 
    #   iteration
yrm.YAMLSetRebalanceByCell(True)            
    # True (default), individual cell times are used in rebalancing; 
    # False, average times are used in rebalancing
# yrm.YAMLUseSolutionDensityVolume(False)     
    # True, use solution density and volume as calculated by PHREEQC; 
    # False, use solution density set by SetDensityUser and the volume 
    #   determined by the product of  SetSaturationUser, SetPorosity, 
    #   and SetRepresentativeVolume
yrm.YAMLSetPartitionUZSolids(False)         
    # True, the fraction of solids and gases available for reaction is 
    #   equal to the saturation; 
    # False (default), all solids and gases are reactive regardless of 
    #   saturation

## Set Units

In [ ]:
# Set concentration units
yrm.YAMLSetUnitsSolution(2)           
    # 1, mg/L; 2, mol/L; 3, kg/kgs
yrm.YAMLSetUnitsPPassemblage(1)       
    # 0, mol/L cell; 1, mol/L water; 2 mol/L rock
yrm.YAMLSetUnitsExchange(1)           
    # 0, mol/L cell; 1, mol/L water; 2 mol/L rock
yrm.YAMLSetUnitsSurface(1)            
    # 0, mol/L cell; 1, mol/L water; 2 mol/L rock
yrm.YAMLSetUnitsGasPhase(1)           
    # 0, mol/L cell; 1, mol/L water; 2 mol/L rock
yrm.YAMLSetUnitsSSassemblage(1)       
    # 0, mol/L cell; 1, mol/L water; 2 mol/L rock
yrm.YAMLSetUnitsKinetics(1)           
    # 0, mol/L cell; 1, mol/L water; 2 mol/L rock

In [ ]:
# Set conversion from seconds to user units (days);
#   Only affects one print statement
time_conversion = 1.0 / 86400.0
# yrm.YAMLSetTimeConversion(time_conversion)      
    # Factor to convert seconds to user time units

## Set Initial Conditions

In [ ]:
# Set these parameters here, 
# to match order in `Mup3d._write_phreeqc_init_file()`

# Set initial porosity
por = [1.0] * nxyz # mf6 handles porosity. Set to 1.
yrm.YAMLSetPorosity(por)                        
    # Porosity, unitless, for each reaction cell as an array of size 
    #   nxyz

In [ ]:
# From `Mup3d._write_phreeqc_init_file()`

print_chemistry_mask = [1]*nxyz
yrm.YAMLSetPrintChemistryMask(print_chemistry_mask)
yrm.YAMLSetPrintChemistryOn(False, True, False)  # workers, initial_phreeqc, utility


In [ ]:
# Set representative volume
rv = [1] * nxyz                                 
yrm.YAMLSetRepresentativeVolume(rv)             
    # Representative volumes, in liters, of each reaction cell as an 
    #   array of size nxyz. Default is 1.0 liter.

# Set initial density
density = [1.0] * nxyz
yrm.YAMLSetDensityUser(density)                 
    # Density for each reaction cell as an array of size nxyz

# Set initial porosity
# por = [0.2] * nxyz
# yrm.YAMLSetPorosity(por)                        
    # Porosity, unitless, for each reaction cell as an array of size 
    #   nxyz

# Set initial saturation
sat = [1] * nxyz
yrm.YAMLSetSaturationUser(sat)                  
    # Saturation fraction for each reaction cell ranging from 0 to 1 as 
    #   an array of size nxyz

## Load Database and Run Input File

In [ ]:
database_path = databasews / 'phreeqc.dat'
database_path.exists()

In [ ]:
# Load database
yrm.YAMLLoadDatabase(str(database_path))          
    # String containing the database name for all IPhreeqc 
    #   instances--workers, InitialPhreeqc, and Utility

# Run file to define solutions and reactants for initial conditions, 
#   selected output
workers = True             
    # Worker instances do the reaction calculations for transport
initial_phreeqc = True     
    # InitialPhreeqc instance accumulates initial and boundary 
    #   conditions
utility = True             
    # Utility instance is available for processing
chemistry_name = str(chemistry_name_filepath)
    # Filepath to the PHREEQC input file (*.pqi)
yrm.YAMLRunFile(workers, initial_phreeqc, utility, chemistry_name)        
    # YAMLRunFile runs a PHREEQC input file. The first three arguments 
    #   determine which IPhreeqc instances will run the file


## Clear contents of Workers and Utility

In [ ]:
# Run PHREEQC input string to delete all inputs
initial_phreeqc = False    
    # False Indicates InitialPhreeqc will not run the string
input = "DELETE; -all"     
    # String containing PHREEQC input that clears contents for the 
    #   instances that run
yrm.YAMLRunString(workers, initial_phreeqc, utility, input)       
    # YAMLRunString runs a PHREEQC input string. The first three 
    #   arguments determine which IPhreeqc instances will run the string

# Allow standard output variables to be available through BMI GetValue
output_vars = "AddOutputVars"                       
    # A string value that includes or excludes variables from 
    #   GetOutputVarNames, GetValue, and other BMI methods
include_vars = "true"                               
    # A string value that can be "false", "true", or a list of items to 
    #   be included as accessible variables
yrm.YAMLAddOutputVars(output_vars, include_vars)    
    # AddOutputVars allows selection of sets of output variables that 
    #   are available through the BMI method GetValue

## Find Components and Transfer Definitions

In [ ]:
# Determine number of components to transport
yrm.YAMLFindComponents()    
    # YAMLFindComponents accumulates a list of elements that have been 
    #   defined in a solution or any other reactant (EQUILIBRIUM_PHASE, 
    #   KINETICS, and others). The list is the set of components that 
    #   needs to be transported

# initial solutions
initial_solutions = [1] * nxyz
yrm.YAMLInitialSolutions2Module(initial_solutions)      
    # SOLUTION definitions, as an array of size nxyz, from the 
    #   InitialPhreeqc instance to be transferred to the reaction-module
    #   workers

# initial exchanges
initial_exchanges = [1] * nxyz
yrm.YAMLInitialExchanges2Module(initial_exchanges)      
    # EXCHANGE definitions, as an array of size nxyz, from the 
    #   InitialPhreeqc instance to be transferred to the reaction-module
    #   workers

TODO: ADD USER_PUNCH from `ex4/mf6rtm/phinp.dat` to .pqi file to configure outputs for plotting
In mf6rtm, this is done by `Mup3d.generate_phreeqc_script()`


In [ ]:
# From `Mup3d._write_phreeqc_init_file()`

yrm.YAMLRunCells()
# Initial equilibration of cells
time = 0.0
yrm.YAMLSetTime(time)

## Write YAML File

In [ ]:
phreeqc_config_filepath = mf6_input_path / "ex6_advect3d_rm.yaml"

In [ ]:
yrm.WriteYAMLDoc(str(phreeqc_config_filepath))      
    # String containing file name where YAML document will be written

In [ ]:
# Confirm that the file was created
phreeqc_config_filepath.exists()

# Run rtm using PhreeqcRM YAML

### Initialize PhreeqcRM interface with selected YAML

In [ ]:
# Initialize phreeqcrm interface
rm = mf6rtm.phreeqcbmi.PhreeqcBMI(str(phreeqc_config_filepath))

### Initilize ModflowAPI interface

In [ ]:
# initialize the interfaces
wd = mf6_input_path
dll = 'libmf6'

mf6 = mf6rtm.mf6api.Mf6API(wd, dll)

In [ ]:
# Save list of outputs for use later
output_var_names = mf6.get_output_var_names()
input_var_names = mf6.get_input_var_names()

In [ ]:
# with concentration data ('concentration')
conc_var_names = [var for var in input_var_names if 'conc' in var]
conc_var_names

In [ ]:
# with concentration data ('X')
gwt_var_names = [var for var in input_var_names if 'GWT_' in var and '/X' in var]
# gwt_var_names

In [ ]:
# mf6.sim.get_model(mf6.sim.model_names[0]).disv

### Initialize & Run MF6RTM coupling interface

In [ ]:
rtm = mf6rtm.mf6rtm.Mf6RTM(wd, mf6, rm)

In [ ]:
### NOTE: comfirm this is correct!
rtm.component_model_dict

In [ ]:
# confirm it matches above, otherwise correct
rtm.component_model_dict == component_gwt_model_dict

In [ ]:
# This function is not implementned for DISV
# mf6.get_grid_rank(1)

In [ ]:
mf6.get_var_rank("GWT_NA/FMI/GWFSAT")

In [ ]:
mf6.get_value(f"GWT_NA/X")

### RTM Solve

In [ ]:
rtm.solve()

FloPy: Elapsed run time:  1.007 Seconds

MF6RTM: Reactive transport solution finished at 2025-05-23 16:18:06 --- it took:   0.098398 mins =  (5.9 seconds)

Coupling to PHREEQC took 5.9x longer!

# Read & Plot Results

## Chemistry Outputs in memory

In [ ]:
headings = rtm.phreeqcbmi.GetSelectedOutputHeadings()
print(headings.shape)
headings

In [ ]:
# number of timesteps 'nstp'
nstp

In [ ]:
rtm.phreeqcbmi.GetSelectedOutput().shape

In [ ]:
rtm._sout_k.shape

In [ ]:
rtm.ctime

In [ ]:
rtm.phreeqcbmi.soutdf

In [ ]:
rtm._current_soutdf

In [ ]:
sout = rtm.phreeqcbmi.GetSelectedOutput()
sout

In [ ]:
i = 0
var_name = headings[i]
print(var_name)
sout[i : i + nxyz]

## Read Output files

In [ ]:
output_filepath = mf6_input_path / 'sout.csv'
output_filepath.exists()

In [ ]:
## Read mf6rtm results
simapi = pd.read_csv(
    output_filepath, 
    sep = ',', 
    skipinitialspace=True, 
    index_col=[0],
)
simapi.info()
simapi.index.name

In [ ]:
simapi.cell.max()

In [ ]:
# create table for last cell representing the column effluent
mf6df = simapi[simapi.cell==simapi.cell.max()]
mf6df

## Plot

In [ ]:
# Create interactiveHoloViz plot
mf6df.plot(
    y=['Na', 'Ca', 'K', 'Cl'], 
    kind='line', 
)

In [ ]:
# Compare to 1D results from Example 4
ex4_plot_filepath = working_dir / 'ex4.png'
if ex4_plot_filepath.exists():
   ex4_plot = Image(filename=ex4_plot_filepath)
else:
   print('Run Example 4 to create ex4.png')
ex4_plot

# END